<a href="https://colab.research.google.com/github/takrim1999/bdset/blob/DL/pretrained_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing Libraries

In [51]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras import layers
import datetime
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard

## Defining Constants

In [42]:
BATCH_SIZE = 16
IMAGE_SIZE = (128,128)
EPOCHS = 10
CHANNELS = 3
SEED = 42
AUTOTUNE = tf.data.AUTOTUNE

## Defining Dataset Path

In [43]:
TRAIN_DIR = "/kaggle/input/dogs-vs-cats/train"
TEST_DIR = "/kaggle/input/dogs-vs-cats/test"

## Image dataset loading via keras dataset loading

In [44]:
train_ds, val_ds = keras.utils.image_dataset_from_directory(
    directory = TRAIN_DIR,
    labels="inferred",
    label_mode="int",
    # class_names=None,
    # color_mode="rgb",
    batch_size= BATCH_SIZE,
    # 32,
    image_size= IMAGE_SIZE,
    #  (256, 256),
    # shuffle=True,
    seed=SEED,
    validation_split=0.2,
    subset='both',
    # interpolation="bilinear",
    # follow_links=False,
    # crop_to_aspect_ratio=False,
    # pad_to_aspect_ratio=False,
    # data_format=None,
    # verbose=True,
)

Found 20000 files belonging to 2 classes.
Using 16000 files for training.
Using 4000 files for validation.


## Performance Optimization

In [45]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

## Preprocessing Layer

In [46]:
data_augmentation = keras.Sequential(
    [
    layers.Rescaling(1./255), # normalization
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.2),
    layers.RandomTranslation(width_factor=0.2, height_factor=0.2)
        ]
    )

print("Preprocessing done")

Preprocessing done


## Loading resnet50 pretrained model

In [47]:
base_model = keras.applications.ResNet50(
    include_top=False, # not taking the dense layer pretrained model to train the model, using my own model
    weights="imagenet",
    # input_tensor=None,
    input_shape=(*IMAGE_SIZE,CHANNELS),
    # pooling=None,
    # classes=1000,
    # classifier_activation="softmax",
    # name="resnet50",
)
print("Model Loaded")

Model Loaded


## Build model

In [48]:
inputs = keras.Input(shape = (*IMAGE_SIZE,CHANNELS))
x = data_augmentation(inputs)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.summary()

Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_11 (InputLayer)     │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_1 (Sequential)       │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resnet50 (Functional)           │ (None, 4, 4, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_5      │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │         2,049 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,589,761 (89.99 MB)

 Trainable params: 23,536,641 (89.79 MB)

 Non-trainable params: 53,120 (207.50 KB)

## Model Compiling

In [49]:
model.compile(
    optimizer='adam',
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

## Callbacks

In [52]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

callbacks = [
    EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True),
    ModelCheckpoint("best_resnet_model.keras", monitor="val_accuracy", save_best_only=True),
    ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=3, min_lr=1e-7),
    TensorBoard(log_dir=log_dir, histogram_freq=1)
]

## Train Model

In [ ]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=callbacks
)

Epoch 1/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 255s 160ms/step - accuracy: 0.6406 - loss: 0.7719 - val_accuracy: 0.7153 - val_loss: 0.5519 - learning_rate: 0.0010
Epoch 2/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 148s 148ms/step - accuracy: 0.7237 - loss: 0.5566 - val_accuracy: 0.6995 - val_loss: 0.8472 - learning_rate: 0.0010
Epoch 3/10
 978/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 135ms/step - accuracy: 0.7670 - loss: 0.4878

# Fine Tuning

In [ ]:
base_model.trainable = True
model.compile(
    optimizer=keras.optimizers.Adam(1e-5),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

history_fine = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=callbacks
)

## Evaluation

In [ ]:
loss, acc = model.evaluate(val_ds)
print("Validation Accuracy:", acc)

## Final model save

In [ ]:
model.save("catdog_resnet50_final.keras")

## Loading Tensorboard

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs/fit

## Prediction

In [ ]:
from keras.preprocessing import image

def predict_image(model_path, img_path, image_size=IMAGE_SIZE):
    model = keras.models.load_model(model_path)

    img = image.load_img(img_path, target_size=image_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

    prediction = model.predict(img_array)
    label = "🐶 Dog" if prediction[0][0] > 0.5 else "🐱 Cat"
    print(f"Prediction: {label}")
    print(f"Confidence Score: {prediction[0][0]:.4f}")

test_img = "/content/cat.jpeg"
predict_image("best_resnet_model.keras", test_img)

## testing

In [ ]:
test_img = "/content/images.webp"
predict_image("best_resnet_model.keras", test_img)